In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

import seaborn as sns

sns.set()

%matplotlib inline

pd.set_option("display.max_columns", None)

In [2]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, average_precision_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [3]:
df = pd.read_csv("Atsuko_11-26-21_all_employees.csv")

In [4]:
# Helper function to split our data
from sklearn.model_selection import train_test_split

# This is our Logit model
from sklearn.linear_model import LogisticRegression

# Helper fuctions to evaluate our model.
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [5]:
df.head()

,RandomID,POSITION_LVL_CHANGE,DEPTCD1,DEPTCD2,DEPTCD3,DEPTCD4,DEPTCD5,DEPTCD6,DEPTCD7,DEPTCD8,DEPTCD9,DEPTCD10,DEPTCD11,RPTG_DIV_MAPPING1,RPTG_DIV_MAPPING2,RPTG_DIV_MAPPING3,RPTG_DIV_MAPPING4,RPTG_DIV_MAPPING5,RPTG_DIV_MAPPING6,RPTG_DIV_MAPPING7,RPTG_DIV_MAPPING8,LOCATION1,LOCATION2,LOCATION3,LOCATION4,LOCATION5,LOCATION6,LOCATION7,LOCATION8,LOCATION9,JOB_FUNCTION1,JOB_FUNCTION2,JOB_FUNCTION3,JOB_FUNCTION4,JOB_FUNCTION5,JOB_FUNCTION6,LGTHSVC_MOST_RECENT,Country Level Cost of Living Index_MEAN,Country Level Traffic Index_MEAN,Country Level Time Index (in minutes)_MEAN,Country Level Time Ex Index_MEAN,Country Level Inefficiencies Index_MEAN,Country Level Unemployment Rate (%)_MEAN,Country Level Corruption Perception Index_MEAN,Country Level Inflation (%)_MEAN,Country Level GDP (nominal) (in USD billion)_MEAN,Country Level Labor Market Risk Index Score_MEAN,Country Level Political Risk Index Score_MEAN,Country Level Power distance_MEAN,Country Level Individualism_MEAN,Country Level Masculinity_MEAN,Country Level Uncertainty avoidance_MEAN,Country Level Long term orientation_MEAN,Country Level Indulgence_MEAN,Month_Year_Most_Recent,left_company,COMPARATIO_MAX_MINUS_MIN,NBR_POS_CH,NBR_JOB_CH,demand_pressure_high_or_low,Year
0,119341,1,800399,803463,803489,803489,803489,803489,803489,803489,803489,803489,803489,Global Operations,WW Human Resources,WW Human Resources,WW Human Resources,WW Human Resources,WW Human Resources,WW Human Resources,WW Human Resources,Peapack New Jersey,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,Global Operations,Global Operations,Global Operations,Global Operations,Global Operations,Global Operations,33.883641,74.220961,177.813644,38.192120,1930.207292,214.946898,0.052323,71.25,0.01875,20591.96325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,May_2021,0,0.072369,4,0,0.0,2021
1,134992,0,800390,800390,800390,800390,800390,800390,800390,800390,800390,800390,800390,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pleasant Prairie W3 - Hospira,USA-WI-Pleasant Prairie W3-HSP,USA - Direct Mail Hospira,USA - Direct Mail Hospira,USA - Direct Mail Hospira,USA - Direct Mail Hospira,USA - Direct Mail Hospira,USA - Direct Mail Hospira,USA - Direct Mail Hospira,Engineering,Engineering,Engineering,Engineering,Engineering,Engineering,27.778234,77.141081,181.426190,39.171905,2043.656667,208.704286,0.043600,74.00,0.02100,19485.40000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,November_2017,0,0.029392,1,0,0.0,2017
2,78068,2,800066,800390,811385,11264385,10061069,10061069,10061069,10061069,10061069,10061069,10061069,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Pfizer Global Supply,Peapack New Jersey,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,USA - NJ - Peapack,Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,14.817248,74.220961,177.813644,38.192120,1930.207292,214.946898,0.052323,71.25,0.01875,20591.96325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,May_2021,0,0.062942,8,0,0.0,2021
3,146210,1,803386,803386,803386,803386,803386,803386,803386,803386,803386,803386,803386,Worldwide Research & Development,WW Research & Development,WW Research & Development,WW Research & Development,WW Research & Development,WW Research & Development,WW Research & Development,WW Research & Development,"Groton, Connecticut",USA - CT - Groton,USA - CT - Groton,USA - CT - Groton,USA - CT - Groton,USA - CT - Groton,USA - CT - Groton,USA - CT - Groton,USA - CT - Groton,Logistics & Supply Chain Mgmt,Logistics & Supply Chain Mgmt,Logistics & Supply Chain

In [6]:
df.shape

(139167, 61)

In [7]:
# List of keys
keyList = ["JOB_FUNCTION6", "RPTG_DIV_MAPPING8", "LOCATION9", "Month_Year_Most_Recent"]
  
# initialize dictionary
d = {}
  
# iterating through the elements of list
for i in keyList:
    d[i] = None
      
for column_name in df.columns:
  col = df[column_name] # we get the column
  if column_name in d:
    stats = df[column_name].unique() # we create a dict to keep the stats for this column
    d[column_name] = stats # we add the stats dict to our result_dict
    
for key,values in d.items():
    l = []
    for v in values:
        l.append(v)
    d[key] = l
    df[key] = df[key].astype('category')
    df[key] = df[key].cat.reorder_categories(d.get(key), ordered=True)
    df[key] = df[key].cat.codes
    #print(df[key])


In [8]:
df = df.drop(columns=['LOCATION1', 'LOCATION2', 'LOCATION3', 'LOCATION4', 'LOCATION5', 'LOCATION6', 'LOCATION7', "LOCATION8", 
                     'JOB_FUNCTION1',
       'JOB_FUNCTION2', 'JOB_FUNCTION3',
       'JOB_FUNCTION4', "JOB_FUNCTION5", 'RPTG_DIV_MAPPING1','RPTG_DIV_MAPPING2',
       'RPTG_DIV_MAPPING3',"RPTG_DIV_MAPPING4","RPTG_DIV_MAPPING5","RPTG_DIV_MAPPING6", "RPTG_DIV_MAPPING7"])

In [9]:
df = df.dropna()

In [10]:
df['left_company'].value_counts()

0    27337
1     5415
Name: left_company, dtype: int64

In [ ]:
### Separate the majority and minority classes to create balanced set
df_minority  = df[df['left_company']==1]
df_majority = df[df['left_company']==0]

### Now, downsamples majority labels equal to the number of samples in the minority class

df_majority = df_majority.sample(len(df_minority), random_state=0)

### concat the majority and minority dataframes
df = pd.concat([df_majority,df_minority])

## Shuffle the dataset to prevent the model from getting biased by similar samples
df = df.sample(frac=1, random_state=0)

In [11]:
# define X & Y

X = df.drop("left_company", axis=1)
y = df['left_company']

In [12]:
# from numpy import mean
# from numpy import std
# from sklearn import metrics
# from sklearn.datasets import make_classification
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.metrics import make_scorer, accuracy_score
# from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, average_precision_score
# from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=40)

In [13]:
def calibration(data, train_pop, target_pop, sampled_train_pop, sampled_target_pop):

    calibrated_data = \
    ((data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)) /
    ((
        (1 - data) * (1 - target_pop / train_pop) / (1 - sampled_target_pop / sampled_train_pop)
     ) +
     (
        data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)
     )))

    return calibrated_data

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)

In [16]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

In [17]:
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=40)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [18]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
#calibration(model_results/probability output array, rows, targetclass label, rows after randomly sampling them out, 500)
calibration(model, len(df), df.groupby(['left_company']).count(), X_train.count(), y_train.count())

In [19]:
accuracy = model.score(X_test, y_test)
print('Accuracy', accuracy)

Accuracy 0.8344897770376538


In [20]:
y_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      9022
           1       0.00      0.00      0.00      1787

    accuracy                           0.83     10809
   macro avg       0.42      0.50      0.45     10809
weighted avg       0.70      0.83      0.76     10809



In [ ]:
y_hat = y_pred
mae = metrics.mean_absolute_error(y_test,y_hat)
mse = metrics.mean_squared_error(y_test,y_hat)

print ("============")
print ("MAE:                ", mae)
print ("RMSE:               ", math.sqrt(mse))
print ("r2:                 ", model.score(X,y))
#print ("feature_importances:",model.feature_importances_)
#print ("n_features:         ",model.n_features_)
#print ("n_outputs:          ",model.n_outputs_)

In [ ]:
AUC = average_precision_score(y_test, y_pred, average='macro')
AUC

In [ ]:
#f1_score(y_test, y_pred)
f1_score 

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, average_precision_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV

In [ ]:
# Used to Predict On New Data

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# fit the model on the whole dataset
model.fit(X, Y)
# predict a multinomial probability distribution
yhat = model.predict_proba([newDataSet])
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % yhat[0])

In [ ]:
# define the multinomial logistic regression model with a default penalty
LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=1.0)

In [ ]:
# Test Penalties
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
 
# get a list of models to evaluate
def get_models():
	models = dict()
	for p in [0.0, 0.0001, 0.001, 0.01, 0.1, 1.0]:
		# create name for model
		key = '%.4f' % p
		# turn off penalty in some cases
		if p == 0.0:
			# no penalty in this case
			models[key] = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='none')
		else:
			models[key] = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=p)
	return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, Y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate the model
	scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores
 
# define dataset
# X, y = 
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	# evaluate the model and collect the scores
	scores = evaluate_model(model, X, Y)
	# store the results
	results.append(scores)
	names.append(name)
	# summarize progress along the way
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))